In [150]:
import pandas as pd
from pathlib  import Path
from sklearn import tree

In [151]:
train_csv = Path('data/raw/train.csv')
test_csv = Path('data/raw/test.csv')

In [152]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
test_df['Survived'] = -1
train_df['is_Test'] = 0
test_df['is_Test'] = 1

In [153]:
concat_df = pd.concat([train_df, test_df])
concat_df = pd.get_dummies(concat_df , dummy_na  = False, drop_first = False,columns=['Sex', 'Ticket', 'Cabin', 'Embarked'])
concat_df['Age'] = concat_df['Age'].fillna(train_df['Age'].mean())
concat_df['Fare'] = concat_df['Fare'].fillna(train_df['Fare'].mean())
del concat_df['Name']
concat_df.to_pickle('data/out/preprocess.pkl')

In [154]:
df = pd.read_pickle('data/out/preprocess.pkl')

In [155]:
train_valid = df.query('is_Test == 0').copy()
test = df.query('is_Test == 1').copy()

In [156]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_valid, test_size=0.2, shuffle=True, random_state=0)

train_X = train.drop(columns=['Survived', 'is_Test'])
train_y = train['Survived']
valid_X = valid.drop(columns=['Survived', 'is_Test'])
valid_y = valid['Survived']
test_X = test.drop(columns=['Survived', 'is_Test'])

In [157]:
clf = tree.DecisionTreeClassifier().fit(train_X, train_y)

In [162]:
accuracy = (clf.predict(valid_X) == valid_y).mean()

In [173]:
import yaml
obj = {'accuracy': accuracy}
with open('data/out/summary.yaml', 'w') as f:
    yaml.dump(obj, f, encoding='utf-8')

In [159]:
test['Survived'] = clf.predict(test_X)

In [160]:
test[['PassengerId', 'Survived']].to_csv('data/out/gender_submission.csv', index=None)